In [ ]:
import pandas as pd 
import os 
import json 
from dotenv import load_dotenv 
import psycopg 
import requests



In [ ]:
load_dotenv()  # this enable getting data form .env file 


#1. What is os.getenv?
#The os library is part of standard Python. getenv stands for "Get Environment Variable."
#Your computer has a "global" list of variables running in the background 
#(like your username or your operating system type). os.getenv("API_KEY") tells Python:
#"Look at the computer's memory and find a variable named 'API_KEY'." 

In [ ]:


API_KEY = os.getenv("API_KEY") 
SEASON = 2024  
LEAGUE_CODE = "PL" 

url = f"https://api.football-data.org/v4/competitions/{LEAGUE_CODE}/standings"

headers = {
    "X-Auth-Token": API_KEY
}

params = {"season": SEASON}


In [ ]:

# 5. Make the Request
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()
    print("Successfully fetched standings!")

else:
    print(f"Error {response.status_code}: {response.text}")

## Extract 

In [ ]:

league=data['standings'][0]['table'] 


### Turn json into DATAFRAME

In [ ]:

league_table = pd.json_normalize(league) #json_normalize is a specialized "factory" function that takes your messy list and 
print(type(league_table))                #returns a clean, ready-to-use DataFrame object.

In [ ]:
league_table.rename(columns={'position' : 'Rank' ,'playedGames':'Played_Games','goalsFor':'goals_for',
'goalsAgainst':'goals_against','goalDifference':'goal_difference','team_id':'id','team.name':'team_name','team.shortName':'short_name'
,'team.tla':'TLA'} ,inplace=True  )

In [ ]:

league_table['season']='2024/2025'

In [ ]:
new_order = ['season','id','name','short_name','TLA','Rank', 'points', 'Played_Games','won','draw','lost','form','goals_for','goals_against'
,'goal_difference'] 
league_table=league_table[new_order] 
league_table

### Load Data to POSTGRESQL

In [ ]:
from sqlalchemy import create_engine 

In [ ]:
engine_url = f'postgresql://{os.getenv("DB_USER")}:{os.getenv("DB_PASSWORD")}@{os.getenv("DB_HOST")}:5432/{os.getenv("DB_NAME")}'

engine = create_engine(engine_url)

In [ ]:
table_name = 'premier_league_standings'

# 2. Load the data
try:
    league_table.to_sql(
        name=table_name, 
        con=engine, 
        if_exists='replace', 
        index=False          
    )
    print(f" Success! Data loaded into the '{table_name}' table.")
except :
    print(" Error loading to Postgres")

### SQLAlchemy is the Translator: It turns your Python/Pandas commands into SQL.
### Psycopg is the Phone Line: It is the driver that carries those SQL commands to the PostgreSQL server.